In [1]:
import Pkg;Pkg.activate(@__DIR__);Pkg.instantiate();

  Activating project at `~/Documents/eth_projects/freertos_sim/test/julia_cf2`


In [2]:
using GLMakie
using PyCall
GLMakie.activate!(inline=false)

In [11]:
points = Observable(Point2f[randn(2)])

fig, ax = scatter(points)
limits!(ax, -4, 4, -4, 4)

fps = 60
nframes = 1200

for i = 1:nframes
    new_point = Point2f(randn(2))
    points[] = push!(points[], new_point)
    sleep(1/fps) # refreshes the display!
end
display(fig)

GLMakie.Screen(...)

# CHannel demo

In [3]:
function producer(a)
    Channel() do ch
        put!(ch, "start")
        for n=1:4
            put!(ch, 2n)
        end
        put!(ch, "stop")
    end
end;

In [4]:
channel_countdown = producer(2)

Channel{Any}(0) (1 item available)

In [5]:
for i in channel_countdown
    println("countdown $i")
end

countdown start
countdown 2
countdown 4
countdown 6
countdown 8
countdown stop


In [6]:
function producer(c::Channel)

    put!(c, "start")
    for n=1:5
        put!(c, 2n)
    end
    put!(c, "stop")
end;

chnl = Channel(producer);

In [ ]:
# put!(chnl, "selva")

In [7]:
for n=1:5
    println(take!(chnl))
end

start
2
4
6
8


## new demo

In [6]:
# Given Channels c1 and c2,
c1 = Channel(32)
c2 = Channel(32)

# add data to c1
for i in 1:3
    put!(c1, i)
end

In [8]:
# and a function `foo` which reads items from c1, processes the item read
# and writes a result to c2,
function foo()
    for i in 1:3
        data = take!(c1)
        result = data*2               # process data
        put!(c2, result)    # write out result
    end
end

foo (generic function with 1 method)

In [10]:
c2

Channel{Any}(32) (3 items available)